# How it works:

The following calls use pywin32 to connect to Outlook while it is running.
This allows Python to tap into the Outlook calendar of the first email account.
Once tapped in, the functions below scan each calendar appointment for specific keywords in the title.
These keywords are based on the categories described in the UNIS Duty sheet for PhD students.

General naming convention to allow for this process:

{Category} - {Subcategory activity} - {WHO} / {description of task}

Please do not include dashes ("-") or forward slashes ("/") as part of the title other than those placed in the general naming convention.

The categories and activities are listed below:

## Teaching

### TA
- Practical support
- Technical support

### Teacher
- Lectures
- Original lectures
- Colloqiuium
- Original colloquium
- Field leader
- Field assistant

## Supervision

### BSc
- Course project
- thesis project

### MSc
- Course project
- Thesis project

## Grading
- report <15p
- report <25p
- report >25p
- exam BSc <4h
- exam BSc <10h
- exam MSc
- oral
- field

## Skills development
- course

## Other
- IT
- hardware
- software
- documentation


In [5]:
import datetime as dt
import win32com.client
import pandas as pd


def get_calendar(begin,end):
    Outlook = win32com.client.Dispatch("Outlook.Application")
    ns = Outlook.GetNamespace("MAPI")
    appts = ns.GetDefaultFolder(9).Items
    calendar = appts
    calendar.IncludeRecurrences = "True"
    calendar.Sort('[Start]')

    restriction = "[Start] >= '" + begin.strftime('%m/%d/%Y') + "' AND [END] <= '" + end.strftime('%m/%d/%Y') + "'"
    calendar = calendar.Restrict(restriction)
    return calendar


def get_appointments(calendar,subject_kw = None,exclude_subject_kw = None, body_kw = None):

    if subject_kw == None:
        appointments = [app for app in calendar]    
    else:
        appointments = [app for app in calendar if subject_kw in app.Subject]

    if exclude_subject_kw != None:
        appointments = [app for app in appointments if exclude_subject_kw not in app.Subject]
    cal_subject = [app.Subject for app in appointments]
    cal_start = [app.Start for app in appointments]
    cal_end = [app.End for app in appointments]
    cal_body = [app.Body for app in appointments]

    df = pd.DataFrame({'subject': cal_subject,
                       'start': [datetime.replace(tzinfo=None) for datetime in cal_start],
                       'end': [datetime.replace(tzinfo=None) for datetime in cal_end],
                       'body': cal_body})
    return df

def classify_appointments(appointments):
    appointments["Category"], appointments["Activity"], appointments["Who"] = appointments.subject.apply(lambda x: x.split(" / ")[0].split(' - ')).values[0]
    return appointments

def analyse_activity(s):
    try:
        description = s.subject.split(" / ")[1]
        s = s.subject.split(" / ")[0]

        cat = s.split(" - ")[0]
        activity = s.split(" - ")[1]
        who = s.split(" - ")[2]
        return cat.lower(), activity.lower(), who, description
    except:
        return None, None, None, "Failed to analyse calendar meeting."

def calculate_time(s):
    try:
        delta = s.end - s.start
        return delta
    except:
        return None

def analyse_calendar(start = dt.datetime(2021,5,26), end = dt.datetime(2021,7,26)):
    cal = get_calendar(start, end)
    analysis = None
    for category in ["Teaching","Supervision","Grading","Skills","Other"]:
        sub_analysis = get_appointments(cal,subject_kw = category)
        
        sub_analysis[["Category","Activity","Who", "Description"]] = sub_analysis.apply(analyse_activity, axis=1, result_type="expand")
        if not sub_analysis.empty:
            sub_analysis = sub_analysis[["Category","Activity","Who", "Description","start","end"]]
            sub_analysis[["time"]] = sub_analysis.apply(calculate_time,axis=1, result_type = "expand")
        
            if analysis is not None:
                analysis = analysis.append(sub_analysis)
            else:
                analysis = sub_analysis.copy()
    
    return analysis

def statistics_calendar(analysis, drop_failed = True):
    if analysis is not None:
        if drop_failed:
            analysis = analysis[analysis["Description"] != "Failed to analyse calendar meeting."]
        return analysis.groupby(by = ["Category","Activity"]).time.sum(), analysis.time.sum()
    


In [6]:
analysis = analyse_calendar(start = dt.datetime(2021,5,26),end = dt.datetime(2021,5,27))
analysis

,Category,Activity,Who,Description,start,end,time
0,teaching,seminar,AG209,Term projects presentations,2021-05-26 10:00:00,2021-05-26 15:30:00,0 days 05:30:00


In [7]:
statistics_calendar(analysis)

(Category  Activity
 teaching  seminar    0 days 05:30:00
 Name: time, dtype: timedelta64[ns],
 Timedelta('0 days 05:30:00'))